In [412]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run Pipeline//processing
%run Pipeline//cleaning
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path
from datetime import datetime as dr
from datetime import date
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')
import pylab as pl
from upload_and_vizualize import camel_to_snake
from datetime import datetime as dt
from datetime import date

In [416]:
'''
date_types = ['year']
date_format = df_dict['violation']['date_format']
date_col = [df_dict['violation']['date_col']]
#print(date_format,date_col)
get_occupied_frame(violation,date_col,date_format,date_types)

date_col = [df_dict['inspection']['date_col']]
get_occupied_frame(inspection,date_col,date_format,date_types)

fce = inspection[['PGM_SYS_ID','STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES','ACTUAL_END_DATE','ACTUAL_END_DATE_year','ACTUAL_END_DATE_datetime']]
violation = violation[['PGM_SYS_ID','AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_DAYZERO_DATE','HPV_DAYZERO_DATE_year','HPV_DAYZERO_DATE_datetime']]
violation = violation[violation.ENF_RESPONSE_POLICY_CODE == 'HPV']
'''

"\ndate_types = ['year']\ndate_format = df_dict['violation']['date_format']\ndate_col = [df_dict['violation']['date_col']]\n#print(date_format,date_col)\nget_occupied_frame(violation,date_col,date_format,date_types)\n\ndate_col = [df_dict['inspection']['date_col']]\nget_occupied_frame(inspection,date_col,date_format,date_types)\n\nfce = inspection[['PGM_SYS_ID','STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES','ACTUAL_END_DATE','ACTUAL_END_DATE_year','ACTUAL_END_DATE_datetime']]\nviolation = violation[['PGM_SYS_ID','AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_DAYZERO_DATE','HPV_DAYZERO_DATE_year','HPV_DAYZERO_DATE_datetime']]\nviolation = violation[violation.ENF_RESPONSE_POLICY_CODE == 'HPV']\n"

In [419]:
inspection.head()

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES
0,MI00000000000A1932,3600154506,E,INS,Inspection/Evaluation,PCE,PCE On-Site,06-09-2015,"CAAFESOP, CAASIP"
1,CASJV00006029S2076,3600154515,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-10-2015,CAATVP
2,CASJV00006039C2587,3600154514,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-08-2015,CAASIP
3,CASJV00006029S1234,3600154512,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-10-2015,CAATVP
4,CASJV00006099N7478,3600154520,L,INS,Inspection/Evaluation,FOO,FCE On-Site,06-11-2015,CAATVP


In [366]:
#### FAILED LABEL #####

In [429]:
def change_to_zero(series_row):
    if type(series_row) != str:
        return 0
    else:
        return 1

def change_to_zero_float(series_row):
    if series_row >= 1:
        return 1
    else:
        return 0
    
def generate_label(start_year, end_year):
    
    violhist = read_file('data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
    fce = read_file('data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')

    
    date_types = ['year']
    date_format = df_dict['violation']['date_format']
    date_col = [df_dict['violation']['date_col']]
    #print(date_format,date_col)
    get_occupied_frame(violhist,date_col,date_format,date_types)

    date_col = [df_dict['inspection']['date_col']]
    get_occupied_frame(fce,date_col,date_format,date_types)

    fce = inspection[['PGM_SYS_ID','STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES','ACTUAL_END_DATE','ACTUAL_END_DATE_year','ACTUAL_END_DATE_datetime']]
    violhist = violhist[['PGM_SYS_ID','AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_DAYZERO_DATE','HPV_DAYZERO_DATE_year','HPV_DAYZERO_DATE_datetime']]
    violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE == 'HPV']
    #removing FRVs
    #violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE != 'FRV']
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] >= start_year]
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] <= end_year]
    
    #for fce
    fce = fce[fce['ACTUAL_END_DATE_year'] >= start_year]
    fce = fce[fce['ACTUAL_END_DATE_year'] <= end_year]
    
    merged_hpv_fce = pd.merge(violhist, fce, how='right', left_on=['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], right_on=['PGM_SYS_ID','ACTUAL_END_DATE'])
    
    merged_hpv_fce['Outcome'] = merged_hpv_fce.ENF_RESPONSE_POLICY_CODE.apply(change_to_zero)
    #finding 0's
    
    output = merged_hpv_fce.groupby(['PGM_SYS_ID', 'ACTUAL_END_DATE_year']).sum().reset_index()
    
    output.Outcome = output.Outcome.apply(change_to_zero_float)
    
    return output, merged_hpv_fce

In [430]:
start_date = '2007'
end_date = '2016'
out, merged = generate_label(start_date, end_date)

In [431]:
out.head()

,PGM_SYS_ID,ACTUAL_END_DATE_year,Outcome
0,020000000003605590,2015,0
1,020000003400500100,2014,0
2,020000003400546143,2010,0
3,020000003400710001,2015,0
4,020000003400900010,2013,0


In [432]:
out.Outcome.value_counts()

0    338010
1      1984
Name: Outcome, dtype: int64

In [400]:
merged.Outcome.value_counts()

0    683079
1      4784
Name: Outcome, dtype: int64

In [414]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {#'data_file': 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv',
                        'interest_var': ['AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [406]:
def generate_features(start_date, end_date):
    #violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)
    
    violation_df = process_violation(violation, start_date, end_date)
    inspection_df = process_titlev(titlev, start_date, end_date)
    stacktest_df = process_stacktest(stacktest, start_date, end_date)
    formalact_df = process_formalact(formalact, start_date, end_date)
    informalact_df = process_informalact(informalact, start_date, end_date)
    noninspectHPV_df = process_noninspectHPV(violation, inspection, start_date, end_date)
    
    final_df = pd.merge(inspection_df, violation_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, stacktest_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, formalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, informalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, noninspectHPV_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])

    re_separate = r'(.[^_]*)_(.*)'
    sep = lambda x: pd.Series([i for i in re.split(re_separate,x)])
    final_id_year = final_df['id_+_date'].apply(sep)
    final_df =pd.concat([final_id_year.rename(columns={1:'PGM_SYS_ID',2:'HPV_DAYZERO_DATE_year'}), final_df], axis=1)
    final_df.drop([0, 3, 'id_+_date'], axis = 1, inplace = True)

    return final_df

In [173]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [413]:
FACILITY = 'data/ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'data/ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'data/ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [407]:
#Checking for feature generation
start_date = '2009'
end_date = '2012'
feat = generate_features(start_date, end_date)
feat.shape

(56246, 173)